<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_diffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0-rc1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 0.25.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
# 求导数例子
def f(x):
    return 3. * x ** 2 + 2. * x - 1
#这里定义一个近似求导数方法，f是函数，x是指在哪一点上的导数，eps足够小的时候，导数就足够真
def approximate_derivative(f, x, eps=1e-3):
    return (f(x + eps) - f(x - eps)) / (2. * eps)

print(approximate_derivative(f, 1.))

7.999999999999119


In [4]:
def g(x1, x2): # 有两个变量，如何求到苏呢
    return (x1 + 5) * (x2 ** 2)

def approximate_gradient(g, x1, x2, eps=1e-3): #这是一个多元函数，
    dg_x1 = approximate_derivative(lambda x: g(x, x2), x1, eps) #x1求偏导，把x2固定住
    dg_x2 = approximate_derivative(lambda x: g(x1, x), x2, eps) #x2求偏导，把x1固定住
    return dg_x1, dg_x2

print(approximate_gradient(g, 2., 3.))
    

(8.999999999993236, 41.999999999994486)


In [5]:
#下面看下在tensorflow里是如何自定义实现导数
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)

#在tensorflow里面求导数，需要用到一个叫tf.GradientTape()的东西，这个东西呢，就可以帮我们进行导数的求解
with tf.GradientTape() as tape:
    # 我打开tf.GradientTape()这个东西之后呢，就可以去定义我们的函数了
    z = g(x1, x2) #这里用的是g这个函数，也就是我把x1,x2传给这个g函数，然后得到一个z
    # 定义好这个函数呢，我们就可以用tape去帮我们求偏导

#下面我们用tape 来求z对于x1的偏导
dz_x1 = tape.gradient(z, x1) #第一个参数是函数的输出，第二个参数是他指定的变量，也就是他对那个变量求偏导
print(dz_x1)

try:
    dz_x2 = tape.gradient(z, x2) #我们也可以用tape去做z对x2的偏导，但是在tensorflow实现里面呢，这个tape是只能用一次的。
    #也就是这个tape调用了一次以后呢，这个对象就会被释放，就不能再用了，所以我们先用try catch呢把异常抓住，后面再看如何解决这个问题
except RuntimeError as ex:
    print(ex)
#从结果中看出，z对于x1的偏导呢，这个值是9
#对于z对于x2求导呢，我们发现他会报出一个异常，这个异常说明，对于没有被保存的tapes, 不能被执行第二次，只能执行一次
#下面看下如何解决这个问题

tf.Tensor(9.0, shape=(), dtype=float32)
GradientTape.gradient can only be called once on non-persistent tapes.


In [6]:
#因为这是一个没有保存的tapes,所以要把它声明称一个可以保存的tapes，persistent = True
#这样就可以对这个tape调用多次，但是呢我们要注意，这个tape呢，用完以后要删掉 del tape
#因为这里呢把persistent设成true了，系统就不会帮你去释放资源了，你需要自己去释放资源
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape(persistent = True) as tape:
    z = g(x1, x2)

dz_x1 = tape.gradient(z, x1)
dz_x2 = tape.gradient(z, x2)
print(dz_x1, dz_x2)

del tape

tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(42.0, shape=(), dtype=float32)


In [7]:
#虽然上面这样，我们可以求出z对于x1 x2的偏导，但是我们毕竟是求了两次
#我们还有两外一种方法，我们可以同时把z关于x1和x2的偏导都求出来
#下面演示该如何实现
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape() as tape:
    z = g(x1, x2)

dz_x1x2 = tape.gradient(z, [x1, x2]) #这里传入的是[x1,x2], 输出的是dz_x1x2

print(dz_x1x2) #这样他返回的偏导呢 也是一个列表，分别是关于x1和x2得偏导

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=42.0>]


In [0]:
#上面是tf.GradientTape对tensorflow里面变量做偏导
#下面试验下 如何对常量求偏导
x1 = tf.constant(2.0)
x2 = tf.constant(3.0)
with tf.GradientTape() as tape:
    z = g(x1, x2)

dz_x1x2 = tape.gradient(z, [x1, x2])

print(dz_x1x2)
#我们发现我们得到得都是None
#那么就完全没办法去关注这个constant得梯度了吗
#不是的，在tensorflow里可以通过一些变动呢去关注这个constant的导数


[None, None]


In [8]:
#那么是什么样的变动呢
#在这里呢，需要做的呢，是我要告诉这个tape呢，我需要关注哪些constant
x1 = tf.constant(2.0)
x2 = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x1) #就是我要关注这两个常量的导数
    tape.watch(x2) #就是我要关注这两个常量的导数，虽然我可能不去计算他，但是呢，还是要关注下
    z = g(x1, x2) 

dz_x1x2 = tape.gradient(z, [x1, x2])

print(dz_x1x2)
#这样就可以得到常量的导数了

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=42.0>]


In [9]:
#这是由多个目标函数下，多个目标函数对一个变量求导的实现
#在上面呢，我们用一个目标函数给两个变量求导数，那能不能我有两个目标函数，然后对一个变量求导数呢
#下面看如何实现
x = tf.Variable(5.0) #定义一个变量，值为5
with tf.GradientTape() as tape: 
    z1 = 3 * x  #定义两个函数z1
    z2 = x ** 2 #定义两个函数z2
tape.gradient([z1, z2], x) #用tape求z1,z2对x的导数
#得到的结果是 z1对于x导数加上z2对于x的导数
#这里z1对x的导数应该是3，z2对x的导数应该是10，合起来就是13
#这是由多个目标函数下，多个目标函数对一个变量求导的实现

<tf.Tensor: shape=(), dtype=float32, numpy=13.0>

In [10]:
#如何求解二阶导数呢
#需要使用一个嵌套的tape
#这里依然使用g作为我们的目标函数来进行求导
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape(persistent=True) as outer_tape: #外层tape,这两个tape都可能调用多次，所以都用了persistent=True
    with tf.GradientTape(persistent=True) as inner_tape: #内层tape,这两个tape都可能调用多次，所以都用了persistent=True
        z = g(x1, x2) #这里定义目标函数
    #在嵌套的内部，就可以求inner_tape的梯度了
    inner_grads = inner_tape.gradient(z, [x1, x2])  #求z对[x1, x2]的偏导

outer_grads = [outer_tape.gradient(inner_grad, [x1, x2]) #到了外层以后呢，我们分别用inner_grad内层求出来的导数呢去对x1,x2求偏导
               for inner_grad in inner_grads]

print(outer_grads) #把外层求导的结果打印出来
del inner_tape
del outer_tape
#得到的是一个2*2的矩阵，
#分别是z对于x1的二阶导数，z先对x2求导再对x1求导
#然后呢右下角的值正好是反过来
#z先对x1求导，再对x2求导 和 z对x2的二阶导数

#如果想求三阶 四阶导数的话，不停的嵌套就可以了，但是平时呢，一般也就求到二阶


[[None, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>], [<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=14.0>]]


In [11]:
#下面模拟实现 梯度下降算法
learning_rate = 0.1
x = tf.Variable(0.0)

for _ in range(100): #进行100步，每一步都求导，然后更新
    with tf.GradientTape() as tape: #用GradientTape去进行求导
        z = f(x) #f(x)作为目标函数
    dz_dx = tape.gradient(z, x) #求导
    x.assign_sub(learning_rate * dz_dx) #x =x -learning_rate*导数
print(x)
#这是一个简单的梯度下降的模拟

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.3333333>


In [12]:
#keras里面用的是optimizer，那么如何和optimizer结合使用呢
#将GradientTape如何和optimizer结合使用
#这里依然将f(x)设为目标函数
learning_rate = 0.1
x = tf.Variable(0.0)

#这里需要使用optimizer.apply_gradients来更新我们的x
optimizer = keras.optimizers.SGD(lr = learning_rate)

for _ in range(100):
    with tf.GradientTape() as tape:
        z = f(x)
    dz_dx = tape.gradient(z, x)
    optimizer.apply_gradients([(dz_dx, x)]) #这里只需要调用apply_gradients就可以啦
    #这里面参数呢，是一个列表，列表里面呢，每个元素都是一个pair.pair里面存储的就是一个变量，以及他的梯度
    #梯度在前，变量在后，在这里呢，我们只有一个变量，所以列表里只有一个元素
print(x)
#大家只要能定义胡目标函数，就可以用tf.GradientTape()去进行求导了

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.3333333>
